In [28]:
import requests

In [29]:
import pandas as pd
import numpy as np

In [30]:
import xlrd

In [31]:
import folium

In [32]:
import lxml

In [33]:
link  = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(link)

In [34]:
df = pd.read_html(link, header=0, na_values = ['Not assigned'])[0]
df.head()

,Postal code,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [35]:
df.dropna(subset=['Borough'], inplace=True)

In [36]:
n_empty_neighborhood = df[df['Neighborhood'].isna()].shape[0]
print('Empty rows in  Neighborhood column: '+ str(n_empty_neighborhood))

Empty rows in  Neighborhood column: 0


In [46]:
df_postal_codes = pd.DataFrame( df.groupby(['Postal code','Borough']).Neighborhood.agg([('Neighborhood', ', '.join)]))
df_postal_codes.reset_index(inplace=True)
df_postal_codes.head()

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [47]:
df.dropna(subset=['Borough'], inplace=True)

In [49]:
n_empty_neighborhood = df[df['Neighborhood'].isna()].shape[0]
print('Number of rows on which Neighborhood column is empty: '+ str(n_empty_neighborhood))

Number of rows on which Neighborhood column is empty: 0


In [50]:
df_postal_codes.shape

(103, 3)

In [51]:
df_postal_codes.to_csv('postal_codes.csv')

In [52]:
df_coordinates = pd.read_csv('Geospatial_Coordinates.csv')
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [53]:
df_neighborhoods = pd.read_csv('postal_codes.csv',index_col=[0])
df_neighborhoods.head()

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [54]:
df_coordinates.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df_neighborhoods.rename(columns={'Postal code': 'PostalCode'}, inplace=True)

In [66]:
df_neighborhoods_with_coordinates = pd.merge(df_neighborhoods, df_coordinates, on='PostalCode')
df_neighborhoods_with_coordinates.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [69]:
df_neighborhoods_with_coordinates.groupby('Borough').count()['Neighborhood']

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Scarborough         17
West Toronto         6
York                 5
Name: Neighborhood, dtype: int64

In [78]:
df_toronto = df_neighborhoods_with_coordinates[df_neighborhoods_with_coordinates['Borough'].str.contains('Toronto')]
df_toronto.reset_index(inplace=True)
df_toronto.drop('index', axis=1, inplace=True)
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,The Danforth West / Riverdale,43.679557,-79.352188
2,M4L,East Toronto,India Bazaar / The Beaches West,43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [79]:
df_toronto.groupby('Borough').count()['Neighborhood']

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
West Toronto         6
Name: Neighborhood, dtype: int64

In [80]:
lat_toronto = df_toronto['Latitude'].mean()
lon_toronto = df_toronto['Longitude'].mean()
print('Location of Toronto: ' + str(lat_toronto) + " " +  str(lon_toronto))

Location of Toronto: 43.66713498717948 -79.38987324871795


In [81]:
boroughs = df_toronto['Borough'].unique().tolist()

In [82]:
borough_color = {}

In [83]:
for borough in boroughs:
    borough_color[borough]= '#%02X%02X%02X' % tuple(np.random.choice(range(256), size=3))

In [85]:
CLIENT_ID = 'KFOW53W5DSWWAUR00CI3TN2JDTSOPTPTYHQNHTTKOK535XEV' 
CLIENT_SECRET = 'ONT1WAI4M4D0HLIAVKLFYHE5GW0WOIYXGO5GTA2O0FVWSL1O' 
VERSION = '20191212' 
LIMIT = 100
radius = 500

In [112]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
               
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
   
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [113]:
toronto_venues = getNearbyVenues(df_toronto['Neighborhood'],df_toronto['Latitude'],df_toronto['Longitude'])

In [114]:
toronto_venues.shape

(1689, 7)

In [115]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,55,55,55,55,55,55
Brockton / Parkdale Village / Exhibition Place,23,23,23,23,23,23
Business reply mail Processing CentrE,16,16,16,16,16,16
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport,17,17,17,17,17,17
Central Bay Street,76,76,76,76,76,76
Christie,18,18,18,18,18,18
Church and Wellesley,81,81,81,81,81,81
Commerce Court / Victoria Hotel,100,100,100,100,100,100
Davisville,35,35,35,35,35,35


In [135]:
toronto_venues['Venue Category'].unique()[:80]

array(['Trail', 'Health Food Store', 'Pub', 'Park', 'Neighborhood',
       'Greek Restaurant', 'Cosmetics Shop', 'Italian Restaurant',
       'Ice Cream Shop', 'Yoga Studio', 'Brewery',
       'Fruit & Vegetable Store', 'Pizza Place', 'Bookstore',
       'Restaurant', 'Dessert Shop', 'Juice Bar', 'Bubble Tea Shop',
       'Spa', 'Diner', 'Grocery Store', 'Furniture / Home Store', 'Café',
       'Coffee Shop', 'Bakery', 'Caribbean Restaurant',
       'Indian Restaurant', 'Frozen Yogurt Shop', 'Lounge',
       'Liquor Store', 'Gym', 'Fish & Chips Shop', 'Fast Food Restaurant',
       'Sushi Restaurant', 'Pet Store', 'Steakhouse', 'Burrito Place',
       'Movie Theater', 'Sandwich Place', 'Board Shop', 'Fish Market',
       'Gay Bar', 'Cheese Shop', 'Middle Eastern Restaurant',
       'Comfort Food Restaurant', 'Thai Restaurant', 'Seafood Restaurant',
       'American Restaurant', 'Stationery Store', 'Coworking Space',
       'Wine Bar', 'Bar', 'Gym / Fitness Center',
       'Latin Americ

In [118]:
to_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
to_onehot['Neighborhoods'] = toronto_venues['Neighborhood'] 

fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]
to_onehot.head()

,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [119]:
to_onehot.shape

(1689, 237)

In [120]:
to_grouped = to_onehot.groupby(["Neighborhoods"]).mean().reset_index()
to_grouped.head()

,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.018182,0.0,0.0,0.000000,0.0,0.000000
1,Brockton / Parkdale Village / Exhibition Place,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.043478
2,Business reply mail Processing CentrE,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.062500
3,CN Tower / King and Spadina / Railway Lands / ...,0.0,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
4,Central Bay Street,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013158,0.0,...,0.0,0.0,0.0,0.0,0.013158,0.0,0.0,0.013158,0.0,0.013158


In [121]:
to_grouped.shape

(39, 237)

In [150]:
len(to_grouped[to_grouped["Restaurant"] > 0])

26

In [151]:
to_restaurant = to_grouped[["Neighborhoods","Restaurant"]]

In [152]:
to_restaurant.head()

,Neighborhoods,Restaurant
0,Berczy Park,0.036364
1,Brockton / Parkdale Village / Exhibition Place,0.043478
2,Business reply mail Processing CentrE,0.062500
3,CN Tower / King and Spadina / Railway Lands / ...,0.000000
4,Central Bay Street,0.000000


In [172]:
from sklearn.cluster import KMeans
k = 5

to_clustering = to_restaurant.drop(["Neighborhoods"], 1)

# clustering
kmeans = KMeans(n_clusters=k, random_state=1)
kmeans.fit_transform(to_clustering)

kmeans.labels_[0:10]

array([4, 4, 2, 3, 3, 2, 4, 2, 0, 3])

In [173]:
to_merged = to_restaurant.copy()

to_merged["Cluster Labels"] = kmeans.labels_

In [174]:
to_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
to_merged.head()

,Neighborhood,Restaurant,Cluster Labels
0,Berczy Park,0.036364,4
1,Brockton / Parkdale Village / Exhibition Place,0.043478,4
2,Business reply mail Processing CentrE,0.062500,2
3,CN Tower / King and Spadina / Railway Lands / ...,0.000000,3
4,Central Bay Street,0.000000,3


In [175]:
to_merged = to_merged.join(toronto_venues.set_index("Neighborhood"), on="Neighborhood")
to_merged.head()

,Neighborhood,Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.036364,4,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
0,Berczy Park,0.036364,4,43.644771,-79.373306,The Keg Steakhouse + Bar - Esplanade,43.646712,-79.374768,Restaurant
0,Berczy Park,0.036364,4,43.644771,-79.373306,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
0,Berczy Park,0.036364,4,43.644771,-79.373306,Meridian Hall,43.646292,-79.376022,Concert Hall
0,Berczy Park,0.036364,4,43.644771,-79.373306,Starbucks,43.644285,-79.369771,Coffee Shop


In [176]:
to_merged.shape

(1689, 9)

In [177]:
to_merged.sort_values(["Cluster Labels"], inplace=True)
to_merged.head()

,Neighborhood,Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
24,Regent Park / Harbourfront,0.022222,0,43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
14,Harbourfront East / Union Station / Toronto Is...,0.030000,0,43.640816,-79.381752,Maple Leaf Square,43.642925,-79.380892,Plaza
14,Harbourfront East / Union Station / Toronto Is...,0.030000,0,43.640816,-79.381752,Harbour Square Park,43.639253,-79.378395,Park
14,Harbourfront East / Union Station / Toronto Is...,0.030000,0,43.640816,-79.381752,e11even,43.642426,-79.381441,New American Restaurant
14,Harbourfront East / Union Station / Toronto Is...,0.030000,0,43.640816,-79.381752,Sharetea,43.640176,-79.379606,Bubble Tea Shop


In [178]:
map_clusters = folium.Map(location=[lat_toronto, lon_toronto],zoom_start=14)

markers_colors={}
markers_colors[0] = 'red'
markers_colors[1] = 'blue'
markers_colors[2] = 'green'
markers_colors[3] = 'cyam'
markers_colors[4] = 'magenta'
for lat, lon, cluster in zip(to_merged['Neighborhood Latitude'], to_merged['Neighborhood Longitude'], to_merged['Cluster Labels']):
    
    
    folium.features.CircleMarker(
        [lat, lon],
        radius=3,
       
        color =markers_colors[cluster],
        fill_color=markers_colors[cluster],
        fill_opacity=0.65).add_to(map_clusters)
       
map_clusters

In [189]:
#k = 0
to_merged.loc[(to_merged['Cluster Labels'] ==0) & (to_merged['Venue Category'] == 'Restaurant') ]

,Neighborhood,Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
14,Harbourfront East / Union Station / Toronto Is...,0.030000,0,43.640816,-79.381752,Steam Whistle's Biergarten,43.640666,-79.385859,Restaurant
14,Harbourfront East / Union Station / Toronto Is...,0.030000,0,43.640816,-79.381752,Sushi Shop,43.643485,-79.380960,Restaurant
14,Harbourfront East / Union Station / Toronto Is...,0.030000,0,43.640816,-79.381752,Kellys Landing,43.645082,-79.383050,Restaurant
13,"Garden District, Ryerson",0.020000,0,43.657162,-79.378937,Jack Astor's Bar & Grill,43.656019,-79.380326,Restaurant
13,"Garden District, Ryerson",0.020000,0,43.657162,-79.378937,JOEY Eaton Centre,43.655404,-79.381929,Restaurant
8,Davisville,0.028571,0,43.704324,-79.388790,Starving Artist,43.701538,-79.387240,Restaurant
28,Runnymede / Swansea,0.022222,0,43.651571,-79.484450,Supper Solved,43.648781,-79.485233,Restaurant
36,The Danforth West / Riverdale,0.023810,0,43.679557,-79.352188,Rikkochez,43.677267,-79.353274,Restaurant
23,Queen's Park / Ontario Provincial Government,0.029412,0,43.662301,-79.389494,Gallery Grill,43.663841,-79.394309,Restaurant
24,Regent Park / Harbourfront,0.022222,0,43.654260,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [181]:
#k = 1
to_merged.loc[(to_merged['Cluster Labels'] ==1) & (to_merged['Venue Category'] == 'Restaurant') ]

,Neighborhood,Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
20,Moore Park / Summerhill East,0.333333,1,43.689574,-79.38316,Reds,43.692816,-79.384504,Restaurant


In [183]:
#k = 2
to_merged.loc[(to_merged['Cluster Labels'] ==2) & (to_merged['Venue Category'] == 'Restaurant') ]

,Neighborhood,Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
25,Richmond / Adelaide / King,0.060000,2,43.650571,-79.384568,The Keg Steakhouse + Bar - York Street,43.649987,-79.384103,Restaurant
22,Parkdale / Roncesvalles,0.076923,2,43.648960,-79.456325,Cider House,43.650688,-79.450685,Restaurant
25,Richmond / Adelaide / King,0.060000,2,43.650571,-79.384568,Bymark,43.647217,-79.381252,Restaurant
25,Richmond / Adelaide / King,0.060000,2,43.650571,-79.384568,Bardi's Steak House,43.646220,-79.383074,Restaurant
16,India Bazaar / The Beaches West,0.052632,2,43.668999,-79.315572,Harvey's,43.666528,-79.315127,Restaurant
25,Richmond / Adelaide / King,0.060000,2,43.650571,-79.384568,Drake One Fifty,43.649590,-79.384557,Restaurant
25,Richmond / Adelaide / King,0.060000,2,43.650571,-79.384568,Bannock,43.652101,-79.381178,Restaurant
25,Richmond / Adelaide / King,0.060000,2,43.650571,-79.384568,Canoe,43.647452,-79.381320,Restaurant
37,Toronto Dominion Centre / Design Exchange,0.060000,2,43.647177,-79.381576,Marché Mövenpick,43.647262,-79.378600,Restaurant
37,Toronto Dominion Centre / Design Exchange,0.060000,2,43.647177,-79.381576,Bymark,43.647217,-79.381252,Restaurant


In [184]:
#k = 3
to_merged.loc[(to_merged['Cluster Labels'] ==3) & (to_merged['Venue Category'] == 'Restaurant') ]

,Neighborhood,Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


In [185]:
#k = 3
to_merged.loc[(to_merged['Cluster Labels'] ==3) & (to_merged['Venue Category'] == 'Restaurant') ]

,Neighborhood,Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


In [186]:
#k = 4
to_merged.loc[(to_merged['Cluster Labels'] ==4) & (to_merged['Venue Category'] == 'Restaurant') ]

,Neighborhood,Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
6,Church and Wellesley,0.037037,4,43.665860,-79.383160,Wish,43.668759,-79.385694,Restaurant
6,Church and Wellesley,0.037037,4,43.665860,-79.383160,O. Noir,43.669145,-79.382505,Restaurant
0,Berczy Park,0.036364,4,43.644771,-79.373306,The Works Gourmet Burger Bistro,43.648742,-79.374142,Restaurant
0,Berczy Park,0.036364,4,43.644771,-79.373306,The Keg Steakhouse + Bar - Esplanade,43.646712,-79.374768,Restaurant
6,Church and Wellesley,0.037037,4,43.665860,-79.383160,Fabarnak,43.666377,-79.380964,Restaurant
1,Brockton / Parkdale Village / Exhibition Place,0.043478,4,43.636847,-79.428191,Vogue Supper Club,43.636951,-79.425446,Restaurant
29,St. James Town,0.040000,4,43.651494,-79.375418,The Carbon Bar,43.653367,-79.374965,Restaurant
29,St. James Town,0.040000,4,43.651494,-79.375418,GEORGE Restaurant,43.653346,-79.374445,Restaurant
29,St. James Town,0.040000,4,43.651494,-79.375418,The Works Gourmet Burger Bistro,43.648742,-79.374142,Restaurant
19,Little Portugal / Trinity,0.040000,4,43.647927,-79.419750,Founder Restaurant & Bar,43.649478,-79.425352,Restaurant


# Conclusion

From the above analysis we can see that in **cluster 2** has **maximum** number of restaurants.

Out of the total 5 Clusters, **Cluster 0** and **Cluster 4** have **proportionate** amount of restaurants.

 In **Cluster 2**, we see a lot of restaurants and **Cluster 1** has only 1 restaurant which is very few. 
 Opening in these areas would result in **strong competition** in one and low attraction due to other restaurants in another.

Hence, we can say that by opening restaurants in
- Cluster 0: Garden District, Ryerson Harbourfront East, Union Station, The Danforth West , Riverdale, Runnymede , Swansea
- Cluster 4:Stn A PO Boxes, St. James Town, Cabbagetown, St. James Town

these areas, one can have enough attraction due to their new restaurants due to already present restaurants and also moderate competition with average number of restaurants in these areas.